In [1]:
# Import these packages
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom as pdicom
import cv2

/usr/local/lib/python3.5/dist-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [7]:
## Load the original csv as labels_df
pth='/media/sumanth/MY DRIVE/'
cs1= pd.read_csv(pth+'calc_case_description_test_set.csv',index_col=False)
cs2= pd.read_csv(pth+'calc_case_description_train_set.csv',index_col=False)
cs3=pd.concat([cs1,cs2])

# There is a trailing line break at the end of each image's path which is removed here
cs3['ROI mask file path']=pth+'DOI/'+cs3['ROI mask file path']

# Create empty dataframe first
labels_df=pd.DataFrame()

# Read & Display images
for x in range(cs3.shape[0]):
    row=cs3.iloc[x]
    imgloc=row['ROI mask file path']
    try:
        img=pdicom.read_file(imgloc,force=True)
        labels_df=labels_df.append(row,ignore_index=True) 
    except:
        pass
print('# of Images present:',labels_df.shape[0])
print('# of Images missing:',cs3.shape[0]-labels_df.shape[0])
print('Total',cs3.shape[0])

# of Images present: 1872
# of Images missing: 0
Total 1872


In [20]:
# Create new CSV with the images stored with it as newcsv and stored as newcsv.pkl
from skimage.transform import rotate

# Calculate mean pixel array
def preprocess_image(path):
    img=pdicom.read_file(path,force=True)
    imgpix=img.pixel_array

    # Scaling 16-bit pixel to 8-bits
    imgpix=((imgpix/65535)*255).astype(np.uint8)
    
    # Subtracting each image with the mean pixel values
   # avg=mean_pixel_array()

    # Resizing image to (224,224)
    imgpix=cv2.resize(imgpix,(224,224))

    # Contrast enhancement
    equ = cv2.equalizeHist(imgpix)
    
    #dilation
    k1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 5))
    k4 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 7))
    dil= cv2.dilate(equ,k1,iterations = 1)
     #store it back in the dicom image
    img.pixel_array=dil
    return dil

newcsv=pd.DataFrame()
ROTATE_LEVELS=[0,90,180,270]
for x in range(labels_df.shape[0]):
    row=labels_df.iloc[x]
    imgloc=row['ROI mask file path']
    img=preprocess_image(imgloc)
    for y in ROTATE_LEVELS:
        if y:
            imgpix=rotate(img,y,preserve_range=True).astype(np.uint8)
        else:
            imgpix=img
        row['pixel_array']=imgpix
#        print(len(np.fromstring(row['pixel_array'],dtype=np.uint8)))
        newcsv=newcsv.append(row,ignore_index=True)

arr=newcsv.pixel_array
avg=arr.apply(lambda x:x.astype(np.uint64)).mean().astype(np.uint8)
newcsv.pixel_array=newcsv.pixel_array-avg    
    
# Saving the dataframe as a pickle file
newcsv.to_pickle('new.pkl')
print("Done!")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/pandas/core/series.py:769: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


Done!


In [21]:
# Load the new csv
newcsv=pd.read_pickle('newcsv.pkl')

In [4]:
# Reading the image
img=pdicom.read_file(li[-5],force=True)
imgpix=img.pixel_array

# Scaling 16-bit pixel to 8-bits
imgpix=((imgpix/65535)*255).astype(np.uint8)

# Resizing image to (224,224)
imgpix=cv2.resize(imgpix,(224,224))

#Contrast enhancement
equ = cv2.equalizeHist(imgpix)

#dilation
k1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 5))
k4 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 7))
dil= cv2.dilate(equ,k1,iterations = 1)

# Stacking all the images together for comparison
res = np.hstack((imgpix,equ,dil)) #stacking images side-by-side
plt.imshow(res)

NameError: name 'li' is not defined

In [ ]:
# Trying out different kernel shapes
k1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 5))
k2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 5))
k3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 5))
k4 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 7))
print(k1,k2,k3,k4,sep="\n")

In [ ]:
hh=newcsv.pixel_array
img=hh[0]

#print('Image Array\n',img)
avg=hh.apply(lambda x:x.astype(np.uint64)).mean().astype(np.uint8)

#print('Mean Pixel Values from the dataset\n',avg)
res=(img-avg).astype(np.uint8)

#print('Image Array after subtracting mean',res)
plt.imshow(np.hstack((img,avg,res)))

In [11]:
arr=newcsv.pixel_array
avg=arr.apply(lambda x:x.astype(np.uint64)).mean().astype(np.uint8)
newcsv1=newcsv.copy()
newcsv1.pixel_array=newcsv.pixel_array-avg

,ROI mask file path,abnormality id,abnormality type,assessment,breast density,calc distribution,calc type,cropped image file path,image file path,image view,left or right breast,pathology,patient_id,pixel_array,subtlety
0,DOI/Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.95...,1.0,calcification,4.0,2.0,CLUSTERED,PUNCTATE-PLEOMORPHIC,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100...,CC,LEFT,BENIGN,P_00038,"[[43, 73, 72, 72, 72, 20, 13, 5, 39, 77, 77, 7...",2.0
1,DOI/Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.95...,1.0,calcification,4.0,2.0,CLUSTERED,PUNCTATE-PLEOMORPHIC,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100...,CC,LEFT,BENIGN,P_00038,"[[81, 79, 78, 40, 66, 103, 103, 102, 102, 102,...",2.0
2,DOI/Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.95...,1.0,calcification,4.0,2.0,CLUSTERED,PUNCTATE-PLEOMORPHIC,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100...,CC,LEFT,BENIGN,P_00038,"[[208, 206, 205, 205, 162, 153, 158, 157, 177,...",2.0
3,DOI/Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.95...,1.0,calcification,4.0,2.0,CLUSTERED,PUNCTATE-PLEOMORPHIC,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100...,CC,LEFT,BENIGN,P_00038,"[[251, 13, 13, 13, 13, 12, 223, 192, 192, 203,...",2.0
4,DOI/Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9...,1.0,calcification,4.0,2.0,CLUSTERED,PUNCTATE-PLEOMORPHIC,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO/1.3.6.1.4.1.9590.10...,MLO,LEFT,BENIGN,P_00038,"[[120, 128, 132, 132, 132, 132, 131, 131, 131,...",2.0
5,DOI/Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9...,1.0,calcification,4.0,2.0,CLUSTERED,PUNCTATE-PLEOMORPHIC,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO/1.3.6.1.4.1.9590.10...,MLO,LEFT,BENIGN,P_00038,"[[108, 106, 97, 78, 78, 78, 78, 72, 63, 61, 54...",2.0
6,DOI/Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9...,1.0,calcification,4.0,2.0,CLUSTERED,PUNCTATE-PLEOMORPHIC,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO/1.3.6.1.4.1.9590.10...,MLO,LEFT,BENIGN,P_00038,"[[139, 137, 136, 136, 137, 138, 138, 137, 137,...",2.0
7,DOI/Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9...,1.0,calcification,4.0,2.0,CLUSTERED,PUNCTATE-PLEOMORPHIC,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO/1.3.6.1.4.1.9590.10...,MLO,LEFT,BENIGN,P_00038,"[[180, 183, 183, 183, 183, 182, 174, 174, 175,...",2.0
8,DOI/Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9...,1.0,calcification,2.0,2.0,NaN,VASCULAR,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,CC,RIGHT,BENIGN_WITHOUT_CALLBACK,P_00038,"[[138, 136, 135, 135, 135, 135, 135, 134, 135,...",5.0
9,DOI/Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9...,1.0,calcification,2.0,2.0,NaN,VASCULAR,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,CC,RIGHT,BENIGN_WITHOUT_CALLBACK,P_00038,"[[136, 134, 133, 133, 133, 133, 132, 131, 131,...",5.0


In [17]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1872 entries, 0 to 1871
Data columns (total 14 columns):
ROI mask file path         1872 non-null object
abnormality id             1872 non-null float64
abnormality type           1872 non-null object
assessment                 1872 non-null float64
breast density             1872 non-null float64
calc distribution          1433 non-null object
calc type                  1848 non-null object
cropped image file path    1872 non-null object
image file path            1872 non-null object
image view                 1872 non-null object
left or right breast       1872 non-null object
pathology                  1872 non-null object
patient_id                 1872 non-null object
subtlety                   1872 non-null float64
dtypes: float64(4), object(10)
memory usage: 204.8+ KB
